In [26]:
import numpy as np
import pandas as pd
import re

In [27]:
def geom_fac(C1,C2,P1,P2):
    """Compute geometric factor, as many formats give resistivities in terms of 
    apparent resistivity. R2 reads in the raw transfer resistances. 
    Parameters
    ----------
    C1: float, np array
        x position of postive current electrode
    C2: float, np array
        x position of negative current electrode
    P1: float, np array
        x position of postive potential electrode
    P2: float, np array
        x position of negative potential electrode
        
    Returns
    -----------
    k: float, np array
        geometric factor to convert transfer resistance into apparent resistivity 
    """
    Rc1p1 = np.abs(C1 - P1)
    Rc2p1 = np.abs(C2 - P1)
    Rc1p2 = np.abs(C1 - P2)
    Rc2p2 = np.abs(C2 - P2)
    
    denom = (1/Rc1p1) - (1/Rc2p1) - (1/Rc1p2) + (1/Rc2p2)
    k = (2*np.pi)/denom
    return k 


In [32]:
def res2invInputParser(file_path):
    """
    Returns info on the electrode geometry and transfer resistances held in the res2dinv input file.
    It looks for the general array format in the .dat file.

    Parameters
    -----------
    file_path : string
         string mapping to the res2inv input file

    Returns
    ----------
    elec : np array
        electrode coordinate matrix in the form | x | y | z |
    df: pandas dataframe
        dataframe which holds the electrode numbers for in feild measurements and
        apparent resistivities (Rho) and transfer resistances

    """
    c1 = np.array(())
    c2 = np.array(())
    p1 = np.array(())
    p2 = np.array(())

    pa = np.array(())
    ip = np.array(())


    f = open(file_path, 'r')  # open file handle for reading
    dump = f.readlines()  # cache file contents into a list
    f.close()  # close file handle, free up resources

    # first find loke's General array format information in the file

    topo_flag = False  # topography flag
    topo_flag_GA = False  # topography flag for integrated topography in general arrays


    idx_oi = 1
    line = dump[idx_oi]
    a_spac = float(line)  # electrode spacing
    idx_oi += 2
    meas_type_flag = 'appRes'  # default
    line = dump[idx_oi]
    num_meas = int(line)
    idx_oi += 1
    line = dump[idx_oi]
    x_location = int(line)
    # x_location = 0 : First electrode location
    # x_location = 1 : Mid-point location
    # x_location = 2 : Surface distance

    idx_oi += 1
    line = dump[idx_oi]
    if int(line) == 0:
        ip_flag = False
        idx_oi += 1
    else:
        ip_flag = True
        idx_oi += 4

    total_x = np.array(())

    data_dict = {'a': [], 'b': [], 'm': [], 'n': [], 'Rho': [], 'ip': [], 'resist': []}
   # print(idx_oi)
    for k in range(num_meas):
        line = dump[idx_oi + k]
        vals = re.findall(r'[-+]?\d*\.\d+|\d+', line)
       # print(vals)
        a = float(vals[1])
        n = (float(vals[2]))
        pa = np.append(pa, float(vals[3]))
        if ip_flag:
            ip = np.append(ip, float(vals[4]))
        if x_location == 0:
            c2 = np.append(c2, np.around((float(vals[0])), 2))
            c1 = np.append(c1, np.around((float(vals[0]) + a), 2))
            p1 = np.append(p1, np.around((float(vals[0]) + a * (1 + n)), 2))
            p2 = np.append(p2, np.around((float(vals[0]) + a * (2 + n)), 2))
        if x_location == 1:
            mid_point = (float(vals[0]))
            c1 = np.append(c1, mid_point - n * a / 2)
            c2 = np.append(c2, mid_point - a * ((n / 2) + 1))
            p1 = np.append(p1, mid_point + n * a / 2)
            p2 = np.append(p2, mid_point + a * ((n / 2) + 1))

    # convert apparent resistivity back in to transfer resistance and vice versa
    K = geom_fac(c1, c2, p1, p2)

    if meas_type_flag == 'appRes':
        data_dict['resist'] = pa / K
        data_dict['Rho'] = pa

    if ip_flag == True:
        data_dict['ip'] = ip
    else:
        data_dict['ip'] = [0] * num_meas

    total_x = np.append(total_x, c1)
    total_x = np.append(total_x, c2)
    total_x = np.append(total_x, p1)
    total_x = np.append(total_x, p2)
    ex_pos = np.unique(total_x)

    largo = len(c1)


    e_idx_c1 = [np.where(ex_pos == c1[i])[0][0] for i in range(largo)]
    e_idx_c1 = np.add(e_idx_c1, 1)
    data_dict['a'] = np.copy(e_idx_c1)

    e_idx_c2 = [np.where(ex_pos == c2[i])[0][0] for i in range(largo)]
    e_idx_c2 = np.add(e_idx_c2, 1)
    data_dict['b'] = np.copy(e_idx_c2)

    e_idx_p1 = [np.where(ex_pos == p1[i])[0][0] for i in range(largo)]
    e_idx_p1 = np.add(e_idx_p1, 1)
    data_dict['m'] = np.copy(e_idx_p1)

    e_idx_p2 = [np.where(ex_pos == p2[i])[0][0] for i in range(largo)]
    e_idx_p2 = np.add(e_idx_p2, 1)
    data_dict['n'] = np.copy(e_idx_p2)

    num_elec = len(ex_pos)



    topo_flag_idx = idx_oi + num_meas
    try:
        int(dump[topo_flag_idx].strip())
    except ValueError:
        topo_flag_idx += 1

    if int(float(dump[topo_flag_idx].strip())) == 2:  # if we have topography then we should read it into the API
        topo_flag = True
        num_elec_topo = int(dump[topo_flag_idx + 1])
        ex_pos_topo = [0] * num_elec_topo
        ez_pos_topo = [0] * num_elec_topo
        ey_pos = [0] * num_elec  # actaully we can't have a y coordinate for 2d data so these will remain as zero
        ez_pos = [0] * num_elec

        for i in range(num_elec_topo):
            e_pos_topo_str = dump[topo_flag_idx + 2 + i].strip()
            e_pos_topo_vals = re.findall(r'[-+]?\d*\.\d+|\d+', e_pos_topo_str)
            ex_pos_topo[i] = float(e_pos_topo_vals[0])
            ez_pos_topo[i] = float(e_pos_topo_vals[1])

        # finding common topography points
        elecdf = pd.DataFrame()
        elecdf['x'] = ex_pos
        elecdf['z_i'] = ez_pos

        elecdf_topo = pd.DataFrame()
        elecdf_topo['x'] = ex_pos_topo
        elecdf_topo['z_topo'] = ez_pos_topo

        if len(elecdf) != len(elecdf_topo):
            elecdf_merged = pd.merge(elecdf.copy(), elecdf_topo.copy(), how='left', on=['x'])
            ez_pos = np.array(elecdf_merged['z_topo'])
        else:
            ex_pos = ex_pos_topo.copy()
            ez_pos = ez_pos_topo.copy()

            # now we have indexed electrode coordinates in ex_pos :)
    if topo_flag_GA:  # if we have integrated topography in the general arrays
        ey_pos = [0] * num_elec  # actaully we can't have a y coordinate for 2d data so these will remain as zero
        ez_pos = elecs_all[:, 1]

    elif not topo_flag:  # then we dont have any topography and the electrode positions are simply given by thier x coordinates
        ey_pos = [0] * num_elec
        ez_pos = [0] * num_elec

    elec = np.column_stack((ex_pos, ey_pos, ez_pos))

    df = pd.DataFrame(data=data_dict)
    df = df[['a', 'b', 'm', 'n', 'Rho',  'ip', 'resist']]

    return elec, df

In [33]:
res2invInputParser("20m.DAT")

(array([[ 0.,  0.,  0.],
        [ 1.,  0.,  0.],
        [ 2.,  0.,  0.],
        [ 3.,  0.,  0.],
        [ 4.,  0.,  0.],
        [ 5.,  0.,  0.],
        [ 6.,  0.,  0.],
        [ 7.,  0.,  0.],
        [ 8.,  0.,  0.],
        [ 9.,  0.,  0.],
        [10.,  0.,  0.],
        [11.,  0.,  0.],
        [12.,  0.,  0.],
        [13.,  0.,  0.],
        [14.,  0.,  0.],
        [15.,  0.,  0.],
        [16.,  0.,  0.],
        [17.,  0.,  0.],
        [18.,  0.,  0.],
        [19.,  0.,  0.],
        [20.,  0.,  0.]]),
       a   b   m   n    Rho  ip    resist
 0     2   1   3   4  44.65   0  2.368756
 1     2   1   4   5  26.38   0  0.349876
 2     2   1   5   6  30.14   0  0.159898
 3     2   1   6   7  18.84   0  0.049975
 4     2   1   7   8  39.56   0  0.059964
 ..   ..  ..  ..  ..    ...  ..       ...
 165  13  10  17  20  16.61   0  0.169943
 166  13  10  18  21  16.89   0  0.109969
 167  14  11  17  20  16.39   0  0.289839
 168  14  11  18  21   6.84   0  0.069983
 169  15  1